In [1]:
import pandas as pd
from sqlalchemy import engine
from pandasql import sqldf
pysqldf=lambda q:sqldf(q, globals())

In [2]:
name_col_department = ["id", "department"]
departments=pd.read_csv("departments.csv",names=name_col_department)
departments

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [4]:
name_col_employees = ["id", "name","datetime","department_id","job_id"]
hired_employees=pd.read_csv("hired_employees.csv",names=name_col_employees)
hired_employees=hired_employees.dropna()
hired_employees['department_id'] = hired_employees['department_id'].astype(int)
hired_employees['job_id'] = hired_employees['job_id'].astype(int)
hired_employees


,id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07T02:48:42Z,2,96
2,3,Lyman Hadye,2021-09-01T23:27:38Z,5,52
3,4,Lotti Crowthe,2021-10-01T13:04:21Z,12,71
4,5,Gretna Lording,2021-10-10T22:22:17Z,6,80
5,6,Marlow Antecki,2021-04-23T23:45:42Z,6,95
...,...,...,...,...,...
1994,1995,Goran Yong,2021-08-22T21:57:53Z,2,83
1995,1996,Cirstoforo Youings,2021-04-01T17:48:42Z,3,23
1996,1997,Wilek Yurkevich,2021-05-04T12:19:50Z,2,132
1997,1998,Jerry Yven,2021-10-03T14:12:50Z,7,100


In [5]:
name_col_jobs = ["id", "job"]
jobs=pd.read_csv("jobs.csv", names=name_col_jobs)
jobs

,id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing
...,...,...
178,179,Software Engineer II
179,180,Statistician IV
180,181,Programmer Analyst I
181,182,Account Representative I


Number of employees hired for each job and department in 2021 divided by quarter. The
table must be ordered alphabetically by department and job.
------------------------------------------------------------------------------------------------------------------
Número de empleados contratados para cada puesto y departamento en 2021 dividido por trimestre. La tabla
tabla debe estar ordenada alfabéticamente por departamento y puesto de trabajo.

In [6]:
q="""
SELECT
    d.department AS Department,
    j.job AS Job,
    SUM(CASE WHEN strftime('%m', h.datetime) BETWEEN '01' AND '03' THEN 1 ELSE 0 END) AS Q1,
    SUM(CASE WHEN strftime('%m', h.datetime) BETWEEN '04' AND '06' THEN 1 ELSE 0 END) AS Q2,
    SUM(CASE WHEN strftime('%m', h.datetime) BETWEEN '07' AND '09' THEN 1 ELSE 0 END) AS Q3,
    SUM(CASE WHEN strftime('%m', h.datetime) BETWEEN '10' AND '12' THEN 1 ELSE 0 END) AS Q4
FROM
    hired_employees h
JOIN
    departments d ON h.department_id = d.id
JOIN
    jobs j ON h.job_id = j.id
WHERE
    strftime('%Y', h.datetime) = '2021'
GROUP BY
    d.department, j.job
ORDER BY
    d.department, j.job;
"""
df_export=pysqldf(q)
df_export

,Department,Job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
...,...,...,...,...,...,...
928,Training,Teacher,0,2,0,0
929,Training,Technical Writer,0,0,1,0
930,Training,VP Product Management,1,0,0,0
931,Training,VP Quality Control,0,1,0,0


In [7]:
df_export.to_csv("requerimiento1.csv")

List of ids, name and number of employees hired of each department that hired more
employees than the mean of employees hired in 2021 for all the departments, ordered
by the number of employees hired (descending).
---------------------------------------------------------------------------------------
Lista de identificadores, nombre y número de empleados contratados de cada departamento que contrató más empleados que la media de empleados contratados en 2021 para todos los departamentos, ordenados por el número de empleados contratados (descendente)